### Calculate wind

In [5]:
import sys
import glob
import xarray as xr
import numpy as np
from dask.distributed import LocalCluster
import time
import dask.array as da 
import os
from IPython.display import clear_output

In [6]:
t0 = time.time()

ens = 'hpp'
bias_correct = 'QM'
start_year = '1991'
end_year = '2010'
var = 'tas'


In [7]:
years = np.arange(int(start_year), int(end_year) +1)
path = "O:/Public/sharing-4270-CERM/VLYMI/CLIMEX2/GlobusDownload/ClimExII_4_Friends/ClimExII_4_Friends/"

files_uas = []
files_vas = []

for year in years:
    year = str(year)
    files_uas = files_uas + sorted(glob.glob(path +'/' + ens + '/' + year + '/uas_' + ens + '*.nc'))[5:10] # Select only June to October
    files_vas = files_vas + sorted(glob.glob(path +'/' + ens + '/' + year + '/vas_' + ens + '*.nc'))[5:10] # Select only June to October


In [8]:
for i, file in enumerate(files_vas):
    clear_output(wait=True)
    print(i)
    if os.path.isfile("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/CLIMEX2/FWI-variables/" + ens + "/wind-" + file[-12:]) == True:
        continue
    ds_vas = xr.open_dataset(file)
    ds_vas = ds_vas.sel(time = ds_vas.time.dt.hour ==12)
    
    ds_uas = xr.open_dataset(files_uas[i])
    ds_uas = ds_uas.sel(time = ds_uas.time.dt.hour ==12)
    ds_uas = ds_uas.assign(vas=(['time','rlat','rlon'],  ds_vas.vas.data))  
    ds_uas = ds_uas.assign(wind=lambda x: np.sqrt(x.uas*x.uas + x.vas*x.vas))
    ds_uas = ds_uas.drop_vars(['uas', 'vas'])
    ds_uas.to_netcdf("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/CLIMEX2/FWI-variables/" + ens + "/wind-" + file[-12:])

99
